# Example 1: A Laminar Channel Flow

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/google-research/swirl-lm/blob/main/swirl_lm/example/swirl_lm_public_demo1_channel.ipynb)


The example in this colab shows how to run a laminar channel flow with Swirl-LM on TPU. It shows how libraries in Swirl-LM is loaded and build, and the key
components to set up a simulation with Swirl-LM.

Note that this colab requires connection to a runtime with TPU. The default
TPU runtime has 8 cores available.

In [ ]:
%%shell
pip uninstall swirl_lm -y
pip install git+https://github.com/google-research/swirl-lm
pip show swirl-lm

PYTHON_PACKAGES=/usr/local/lib/python3.7/dist-packages
proto_names=(
    'base/parameters.proto'
    'boundary_condition/boundary_conditions.proto'
    'boundary_condition/boundary_models.proto'
    'boundary_condition/immersed_boundary_method.proto'
    'boundary_condition/monin_obukhov_similarity_theory.proto'
    'boundary_condition/rayleigh_damping_layer.proto'
    'equations/pressure.proto'
    'equations/scalars.proto'
    'linalg/poisson_solver.proto'
    'numerics/numerics.proto'
    'physics/combustion/combustion.proto'
    'physics/combustion/wood.proto'
    'physics/thermodynamics/thermodynamics.proto'
    'utility/grid_parametrization.proto'
    'utility/monitor.proto'
    'utility/probe.proto'
)

for proto in ${proto_names[@]}
do
    protoc -I=$PYTHON_PACKAGES --python_out=$PYTHON_PACKAGES \
    $PYTHON_PACKAGES/swirl_lm/$proto
done


In [ ]:
import sys

from absl import flags
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from swirl_lm.base import driver
from swirl_lm.base import driver_tpu
from swirl_lm.base import parameters
from swirl_lm.physics.combustion import turbulent_kinetic_energy
from swirl_lm.utility import tpu_util

FLAGS = flags.FLAGS


# Simulation Setup
To initialize a simulation, one needs to provide the following:
   * A text protobuf file that specifies the solver parameters and physical conditions of a simulation, e.g. boundary conditions
   * A list of flags that specifies the simulation configuration (e.g. domain
   size, partitions) and user defined parameters for a simulation
   * A function that initializes state variables to start the simulation

In [ ]:
# Creates the text protobuf.
pbtxt = (
    '# proto-file: swirl_lm/base/parameters.proto \n'
    '# proto-message: SwirlLMParameters \n'
    ' \n'
    'solver_procedure: VARIABLE_DENSITY \n'
    'convection_scheme: CONVECTION_SCHEME_QUICK \n'
    'time_integration_scheme: TIME_SCHEME_CN_EXPLICIT_ITERATION \n'
    'periodic { \n'
    '  dim_0: false \n'
    '  dim_1: false \n'
    '  dim_2: true \n'
    '} \n'
    'pressure { \n'
    '  solver { \n'
    '    jacobi { \n'
    '      max_iterations: 10 halo_width: 2 omega: 0.67 \n'
    '    } \n'
    '  } \n'
    '  num_d_rho_filter: 3 \n'
    '  update_p_bc_by_flow: true \n'
    '} \n'
    'thermodynamics { \n'
    '  constant_density {} \n'
    '} \n'
    'density: 1.0 \n'
    'kinematic_viscosity: 0.01 \n'
    'boundary_conditions { \n'
    '  name: "u" \n'
    '  boundary_info { \n'
    '    dim: 0 \n'
    '    location: 0 \n'
    '    type: BC_TYPE_DIRICHLET \n'
    '    value: 1.0 \n'
    '  } \n'
    '  boundary_info { \n'
    '    dim: 0 \n'
    '    location: 1 \n'
    '    type: BC_TYPE_NEUMANN \n'
    '    value: 0.0 \n'
    '  } \n'
    '  boundary_info { \n'
    '    dim: 1 \n'
    '    location: 0 \n'
    '    type: BC_TYPE_DIRICHLET \n'
    '    value: 0.0 \n'
    '  } \n'
    '  boundary_info { \n'
    '    dim: 1 \n'
    '    location: 1 \n'
    '    type: BC_TYPE_DIRICHLET \n'
    '    value: 0.0 \n'
    '  } \n'
    '} \n'
    'boundary_conditions { \n'
    '  name: "v" \n'
    '  boundary_info { \n'
    '    dim: 0 \n'
    '    location: 0 \n'
    '    type: BC_TYPE_DIRICHLET \n'
    '    value: 0.0 \n'
    '  } \n'
    '  boundary_info { \n'
    '    dim: 0 \n'
    '    location: 1 \n'
    '    type: BC_TYPE_NEUMANN \n'
    '    value: 0.0 \n'
    '  } \n'
    '  boundary_info { \n'
    '    dim: 1 \n'
    '    location: 0 \n'
    '    type: BC_TYPE_DIRICHLET \n'
    '    value: 0.0 \n'
    '  } \n'
    '  boundary_info { \n'
    '    dim: 1 \n'
    '    location: 1 \n'
    '    type: BC_TYPE_DIRICHLET \n'
    '    value: 0.0 \n'
    '  } \n'
    '} \n'
    'boundary_conditions { \n'
    '  name: "w" \n'
    '  boundary_info { \n'
    '    dim: 0 \n'
    '    location: 0 \n'
    '    type: BC_TYPE_DIRICHLET \n'
    '    value: 0.0 \n'
    '  } \n'
    '  boundary_info { \n'
    '    dim: 0 \n'
    '    location: 1 \n'
    '    type: BC_TYPE_NEUMANN \n'
    '    value: 0.0 \n'
    '  } \n'
    '  boundary_info { \n'
    '    dim: 1 \n'
    '    location: 0 \n'
    '    type: BC_TYPE_DIRICHLET \n'
    '    value: 0.0 \n'
    '  } \n'
    '  boundary_info { \n'
    '    dim: 1 \n'
    '    location: 1 \n'
    '    type: BC_TYPE_DIRICHLET \n'
    '    value: 0.0 \n'
    '  } \n'
    '} \n'
)

with open('channel.pbtxt', 'w') as f:
  f.write(pbtxt)
  

In [ ]:
# Prepares the simulation flags. 
flags.DEFINE_string('f', '', 'kernel')

# The number of cores in 3 dimensions.
FLAGS.cx = 4
FLAGS.cy = 2
FLAGS.cz = 1

# The number of grid points per core in 3 dimensions including ghost cells
# (halos).
FLAGS.nx = 128
FLAGS.ny = 64
FLAGS.nz = 8

# The physical size of the simulation domain in units of m.
FLAGS.lx = 4.0
FLAGS.ly = 1.0
FLAGS.lz = 0.01

# The time step size in units of s.
FLAGS.dt = 1e-3

# The width of the ghost cells on each side of the domain. It is set to 2
# considering the stencil width of the QUICK scheme.
FLAGS.halo_width = 2

# The size of the convolution kernel to be used for fundamental numerical
# operations.
FLAGS.kernel_size = 16

# The number of time steps to run before restart data files are written. The
# completion of these steps is considered as a cycle.
FLAGS.num_steps = 100

# The number of cycles to run in this simulation. It determines the number of
# snapshots to save as restart data files. Note that this parameter is not used
# in this demo.
FLAGS.num_cycles = 1

# The prefix of restart files to be written out.
FLAGS.data_dump_prefix = './channel/channel'

# The prefix of restart files to be loaded as initial conditions.
FLAGS.data_load_prefix = './channel/channel'

# The full path of the simulation configuration file.
FLAGS.config_filepath = './channel.pbtxt'

FLAGS(sys.argv)


In [ ]:
# Defines the function that initializes state variables.

def init_fn_channel(replica_id, coordinates):
  """Initializes state variables in a channel flow."""
  del coordinates

  return {
      'replica_id': replica_id,
      'rho': tf.ones((FLAGS.nz, FLAGS.nx, FLAGS.ny), dtype=tf.float32),
      'u': tf.ones((FLAGS.nz, FLAGS.nx, FLAGS.ny), dtype=tf.float32),
      'v': tf.zeros((FLAGS.nz, FLAGS.nx, FLAGS.ny), dtype=tf.float32),
      'w': tf.zeros((FLAGS.nz, FLAGS.nx, FLAGS.ny), dtype=tf.float32),
      'p': tf.zeros((FLAGS.nz, FLAGS.nx, FLAGS.ny), dtype=tf.float32),
   }


In [ ]:
# Prepares the simulation configuration.
params = parameters.params_from_config_file_flag()


# TPU initialization

In [ ]:
# Initializes the TPU strategy.
computation_shape = np.array([params.cx, params.cy, params.cz])

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException(
      'ERROR: Not connected to a TPU runtime; please see the previous cell in '
      'this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
topology = tf.tpu.experimental.initialize_tpu_system(tpu)
device_assignment, _ = tpu_util.tpu_device_assignment(
      computation_shape=computation_shape, tpu_topology=topology)
tpu_strategy = tf.distribute.experimental.TPUStrategy(
    tpu, device_assignment=device_assignment)
logical_coordinates = tpu_util.grid_coordinates(computation_shape).tolist()

print("All devices: ", tf.config.list_logical_devices('TPU'))

# Run the simulation

In [ ]:
# initializes the simulation.
state = driver_tpu.distribute_values(
      tpu_strategy, value_fn=init_fn_channel,
      logical_coordinates=logical_coordinates)
      

In [ ]:
# Runs the simulation for one cycle.
step_id = tf.constant(0)

state = driver._one_cycle(
    strategy=tpu_strategy,
    init_state=state,
    init_step_id=step_id,
    num_steps=FLAGS.num_steps,
    params=params)

# Post-processing

In [ ]:
# Utility functions for postprocessing.

def merge_result(values, coordinates, halo_width):
  """Merges results from multiple TPU replicas following the topology."""
  if len(values) != len(coordinates):
    raise(
        ValueError,
        f'The length of `value` and `coordinates` must equal. Now `values` has '
        f'length {len(values)}, but `coordinates` has length '
        f'{len(coordinates)}.')

  # The results are oriented in order z-x-y.
  nz, nx, ny = values[0].shape
  nz_0, nx_0, ny_0 = [n - 2 * halo_width for n in (nz, nx, ny)]

  # The topology is oriented in order x-y-z.
  cx, cy, cz = np.array(np.max(coordinates, axis=0)) + 1

  # Compute the total size without ghost cells/halos.
  shape = [n * c for n, c in zip([nz_0, nx_0, ny_0], [cz, cx, cy])]

  result = np.empty(shape, dtype=np.float32)

  for replica in range(len(coordinates)):
    s = np.roll(
        [c * n for c, n in zip(coordinates[replica], (nx_0, ny_0, nz_0))],
        shift=1)
    e = [s_i + n for s_i, n in zip(s, (nz_0, nx_0, ny_0))]
    result[s[0]:e[0], s[1]:e[1], s[2]:e[2]] = (
        values[replica][halo_width:nz_0 + halo_width,
                        halo_width:nx_0 + halo_width,
                        halo_width:ny_0 + halo_width])

  return result



In [ ]:
#@title Results visualization
varname = 'u'  #@param ['u', 'v', 'w', 'p', 'rho']

result = merge_result(
    state[varname].values, logical_coordinates, FLAGS.halo_width)

nx = (FLAGS.nx - 2 * FLAGS.halo_width) * FLAGS.cx
ny = (FLAGS.ny - 2 * FLAGS.halo_width) * FLAGS.cy
nz = (FLAGS.nz - 2 * FLAGS.halo_width) * FLAGS.cz

x = np.linspace(0.0, FLAGS.lx, nx)
y = np.linspace(0.0, FLAGS.ly, ny)
z = np.linspace(0.0, FLAGS.lz, nz)

fig, ax = plt.subplots(figsize=(18, 6))
c = ax.contourf(x, y, result[nz // 2, ...].transpose(), cmap='jet', levels=21)
fig.colorbar(c)
ax.axis('equal')
plt.show()
